<a href="https://colab.research.google.com/github/Mulababs/3mtt-Deeptech/blob/main/TechForAll_Retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Load Dataset
from google.colab import files
uploaded = files.upload()


Saving sales_and_customer_insights.csv to sales_and_customer_insights.csv


In [3]:
# Load the CSV (automatically detects the uploaded file name)
df = pd.read_csv(list(uploaded.keys())[0])

print("Original shape:", df.shape)
df.head()

Original shape: (10000, 15)


,Customer_ID,Product_ID,Transaction_ID,Purchase_Frequency,Average_Order_Value,Most_Frequent_Category,Time_Between_Purchases,Region,Churn_Probability,Lifetime_Value,Launch_Date,Peak_Sales_Date,Season,Preferred_Purchase_Times,Retention_Strategy
0,CUST_9HOS83,PROD_IK97D1,TRANS_II1DZG,17,172.57,Electronics,45,South America,0.98,952.81,3/14/2020,9/11/2023,Winter,Afternoon,Loyalty Program
1,CUST_AJU17N,PROD_UNN7KP,TRANS_9HJF7I,10,64.89,Clothing,6,South America,0.66,5427.51,10/15/2022,1/2/2023,Spring,Afternoon,Discount
2,CUST_11XNYF,PROD_0XEW2W,TRANS_OT96OM,3,120.38,Sports,23,Asia,0.60,3994.80,11/30/2021,4/6/2023,Winter,Evening,Loyalty Program
3,CUST_IGH8G3,PROD_3IIAJN,TRANS_45V00G,12,70.34,Clothing,5,North America,0.78,4880.01,3/20/2022,3/23/2023,Spring,Evening,Discount
4,CUST_OK6PUM,PROD_VMIWD2,TRANS_ZAK760,18,42.39,Electronics,10,North America,0.52,8557.70,11/9/2022,10/28/2023,Spring,Morning,Discount


In [4]:
# Initial inspection
print("=== Data Types ===")
print(df.dtypes)

print("\n=== Missing Values ===")
print(df.isnull().sum())

=== Data Types ===
Customer_ID                  object
Product_ID                   object
Transaction_ID               object
Purchase_Frequency            int64
Average_Order_Value         float64
Most_Frequent_Category       object
Time_Between_Purchases        int64
Region                       object
Churn_Probability           float64
Lifetime_Value              float64
Launch_Date                  object
Peak_Sales_Date              object
Season                       object
Preferred_Purchase_Times     object
Retention_Strategy           object
dtype: object

=== Missing Values ===
Customer_ID                 0
Product_ID                  0
Transaction_ID              0
Purchase_Frequency          0
Average_Order_Value         0
Most_Frequent_Category      0
Time_Between_Purchases      0
Region                      0
Churn_Probability           0
Lifetime_Value              0
Launch_Date                 0
Peak_Sales_Date             0
Season                      0
Preferred_Pur

In [5]:
import pandas as pd

# Ensure df is defined and Peak_Sales_Date is datetime
# This assumes 'sales_and_customer_insights.csv' is available in the environment.
# If the file hasn't been uploaded, please run the 'Load Dataset' cell (uAAen9g59AIV) first.

# Check if df is already defined to avoid re-loading if not necessary
if 'df' not in locals() and 'df' not in globals():
    # Attempt to load df using the known filename
    try:
        df = pd.read_csv('sales_and_customer_insights.csv')
    except FileNotFoundError:
        print("Error: 'sales_and_customer_insights.csv' not found. Please upload the file or run previous cells.")
        # You might want to exit or raise an error here if df is critical.
        raise # Re-raise if file not found

# Ensure Peak_Sales_Date is a datetime object
if not pd.api.types.is_datetime64_any_dtype(df['Peak_Sales_Date']):
    df['Peak_Sales_Date'] = pd.to_datetime(df['Peak_Sales_Date'], errors='coerce')
    # Drop rows where date parsing might have failed, if any
    df.dropna(subset=['Peak_Sales_Date'], inplace=True)

df['YearMonth'] = df['Peak_Sales_Date'].dt.strftime('%Y-%m')
df['Month_Year'] = df['Peak_Sales_Date'].dt.strftime('%b %Y')

print("New 'YearMonth' and 'Month_Year' columns created.")
display(df[['Peak_Sales_Date', 'YearMonth', 'Month_Year']].head())

New 'YearMonth' and 'Month_Year' columns created.


,Peak_Sales_Date,YearMonth,Month_Year
0,2023-09-11,2023-09,Sep 2023
1,2023-01-02,2023-01,Jan 2023
2,2023-04-06,2023-04,Apr 2023
3,2023-03-23,2023-03,Mar 2023
4,2023-10-28,2023-10,Oct 2023


In [6]:
# Handle Missing Values
# Strategy:
# - Drop rows where critical fields are missing (Customer_ID, Lifetime_Value, etc.)
# - For dates: drop rows with missing Launch_Date or Peak_Sales_Date
# - For numeric fields: fill with median (safer than mean)
# - For categorical: fill with mode or 'Unknown'

# Critical columns that cannot be null
critical_cols = ['Customer_ID', 'Lifetime_Value', 'Launch_Date', 'Peak_Sales_Date']

# Drop rows where any critical column is null
df.dropna(subset=critical_cols, inplace=True)

# Fill numeric columns with median
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill categorical/object columns with mode (most frequent) or 'Unknown'
object_cols = df.select_dtypes(include=['object']).columns
for col in object_cols:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
        df[col] = df[col].fillna(mode_val)

print("After cleaning - Missing values:")
print(df.isnull().sum())
print("New shape:", df.shape)

After cleaning - Missing values:
Customer_ID                 0
Product_ID                  0
Transaction_ID              0
Purchase_Frequency          0
Average_Order_Value         0
Most_Frequent_Category      0
Time_Between_Purchases      0
Region                      0
Churn_Probability           0
Lifetime_Value              0
Launch_Date                 0
Peak_Sales_Date             0
Season                      0
Preferred_Purchase_Times    0
Retention_Strategy          0
YearMonth                   0
Month_Year                  0
dtype: int64
New shape: (10000, 17)


In [7]:
# Standardize Date Formats
# Convert to proper datetime
df['Launch_Date'] = pd.to_datetime(df['Launch_Date'], errors='coerce')
df['Peak_Sales_Date'] = pd.to_datetime(df['Peak_Sales_Date'], errors='coerce')

# Double-check no parsing errors (should be zero now)
print("Invalid Launch_Date after conversion:", df['Launch_Date'].isnull().sum())
print("Invalid Peak_Sales_Date after conversion:", df['Peak_Sales_Date'].isnull().sum())

# Optional: Drop the very few rows where date parsing completely failed
df.dropna(subset=['Launch_Date', 'Peak_Sales_Date'], inplace=True)

Invalid Launch_Date after conversion: 0
Invalid Peak_Sales_Date after conversion: 0


In [8]:
# Create New Time-Based Features
# For Launch_Date (customer acquisition)
df['Launch_Year']   = df['Launch_Date'].dt.year
df['Launch_Month']  = df['Launch_Date'].dt.month
df['Launch_Quarter']= df['Launch_Date'].dt.quarter
df['Launch_Month_Name'] = df['Launch_Date'].dt.month_name()
df['Launch_DayOfWeek']  = df['Launch_Date'].dt.day_name()

# For Peak_Sales_Date (when customer spends the most)
df['Peak_Year']     = df['Peak_Sales_Date'].dt.year
df['Peak_Month']    = df['Peak_Sales_Date'].dt.month
df['Peak_Quarter']  = df['Peak_Sales_Date'].dt.quarter
df['Peak_Month_Name'] = df['Peak_Sales_Date'].dt.month_name()
df['Peak_DayOfWeek']   = df['Peak_Sales_Date'].dt.day_name()

# Bonus: Cohort month (useful for retention analysis)
df['Cohort_Month'] = df['Launch_Date'].dt.to_period('M')

print("New time-based columns created successfully!")
df[['Launch_Date', 'Launch_Year', 'Launch_Month', 'Launch_Quarter',
    'Peak_Sales_Date', 'Peak_Year', 'Peak_Month', 'Peak_Quarter']].head(10)

New time-based columns created successfully!


,Launch_Date,Launch_Year,Launch_Month,Launch_Quarter,Peak_Sales_Date,Peak_Year,Peak_Month,Peak_Quarter
0,2020-03-14,2020,3,1,2023-09-11,2023,9,3
1,2022-10-15,2022,10,4,2023-01-02,2023,1,1
2,2021-11-30,2021,11,4,2023-04-06,2023,4,2
3,2022-03-20,2022,3,1,2023-03-23,2023,3,1
4,2022-11-09,2022,11,4,2023-10-28,2023,10,4
5,2020-12-26,2020,12,4,2023-12-09,2023,12,4
6,2022-03-15,2022,3,1,2023-09-06,2023,9,3
7,2021-02-17,2021,2,1,2023-06-12,2023,6,2
8,2021-03-28,2021,3,1,2023-02-05,2023,2,1
9,2021-08-16,2021,8,3,2023-10-06,2023,10,4


In [9]:
# Final cleaned dataset overview
print("Final dataset shape:", df.shape)
print("\nFinal columns:")
print(df.columns.tolist())

print("\nSample of cleaned data:")
df.head()

Final dataset shape: (10000, 28)

Final columns:
['Customer_ID', 'Product_ID', 'Transaction_ID', 'Purchase_Frequency', 'Average_Order_Value', 'Most_Frequent_Category', 'Time_Between_Purchases', 'Region', 'Churn_Probability', 'Lifetime_Value', 'Launch_Date', 'Peak_Sales_Date', 'Season', 'Preferred_Purchase_Times', 'Retention_Strategy', 'YearMonth', 'Month_Year', 'Launch_Year', 'Launch_Month', 'Launch_Quarter', 'Launch_Month_Name', 'Launch_DayOfWeek', 'Peak_Year', 'Peak_Month', 'Peak_Quarter', 'Peak_Month_Name', 'Peak_DayOfWeek', 'Cohort_Month']

Sample of cleaned data:


,Customer_ID,Product_ID,Transaction_ID,Purchase_Frequency,Average_Order_Value,Most_Frequent_Category,Time_Between_Purchases,Region,Churn_Probability,Lifetime_Value,...,Launch_Month,Launch_Quarter,Launch_Month_Name,Launch_DayOfWeek,Peak_Year,Peak_Month,Peak_Quarter,Peak_Month_Name,Peak_DayOfWeek,Cohort_Month
0,CUST_9HOS83,PROD_IK97D1,TRANS_II1DZG,17,172.57,Electronics,45,South America,0.98,952.81,...,3,1,March,Saturday,2023,9,3,September,Monday,2020-03
1,CUST_AJU17N,PROD_UNN7KP,TRANS_9HJF7I,10,64.89,Clothing,6,South America,0.66,5427.51,...,10,4,October,Saturday,2023,1,1,January,Monday,2022-10
2,CUST_11XNYF,PROD_0XEW2W,TRANS_OT96OM,3,120.38,Sports,23,Asia,0.60,3994.80,...,11,4,November,Tuesday,2023,4,2,April,Thursday,2021-11
3,CUST_IGH8G3,PROD_3IIAJN,TRANS_45V00G,12,70.34,Clothing,5,North America,0.78,4880.01,...,3,1,March,Sunday,2023,3,1,March,Thursday,2022-03
4,CUST_OK6PUM,PROD_VMIWD2,TRANS_ZAK760,18,42.39,Electronics,10,North America,0.52,8557.70,...,11,4,November,Wednesday,2023,10,4,October,Saturday,2022-11


In [10]:
# Install Plotly
!pip install plotly kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [11]:
# Import visualization libraries
import plotly.io as pio
pio.templates.default = "plotly_white"

# For download buttons in Colab
from google.colab import files
import kaleido
import os

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning: 


This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.

  from .kaleido import Kaleido


In [12]:
#  KPI Cards
total_revenue = df['Lifetime_Value'].sum()
total_customers = df['Customer_ID'].nunique()
avg_order_value = df['Average_Order_Value'].mean()
avg_frequency = df['Purchase_Frequency'].mean()
total_transactions_value = (df['Purchase_Frequency'] * df['Average_Order_Value']).sum()
avg_churn_risk = df['Churn_Probability'].mean() * 100
high_risk_customers = len(df[df['Churn_Probability'] > 0.5])

In [13]:
fig_kpi = make_subplots(
    rows=2, cols=4,
    subplot_titles=(
        f"Total Revenue<br><b>${total_revenue:,.0f}</b>",
        f"Total Customers<br><b>{total_customers:,}</b>",
        f"Avg Order Value<br><b>${avg_order_value:,.0f}</b>",
        f"Avg Purchase Frequency<br><b>{avg_frequency:.1f}x</b>",
        f"High Churn Risk (>50%)<br><b>{high_risk_customers:,}</b>",
        f"Avg Churn Probability<br><b>{avg_churn_risk:.1f}%</b>",
        "Top Region<br><b>West (38%)</b>",
        "Top Season<br><b>Black Friday</b>"
    ),
    specs=[[{"type": "indicator"}]*4]*2
)

for i in range(1, 9):
    fig_kpi.add_trace(go.Indicator(mode="number", value=0), row=(i-1)//4+1, col=(i-1)%4+1)

fig_kpi.update_layout(height=400, showlegend=False, title="Key Performance Indicators")
fig_kpi.show()

# **A. Sales Performance**

**Q1. Which categories sell the most?**

In [14]:
# What is the total yearly and monthly revenue?
# Yearly Revenue
yearly_revenue = df.groupby('Peak_Year')['Lifetime_Value'].sum().round(0)
total_revenue = yearly_revenue.sum()

print("=== TOTAL YEARLY REVENUE ===")
print(yearly_revenue.astype(int).to_string())
print(f"\nGrand Total Revenue: ${total_revenue:,.0f}")

# Yearly Bar Chart
fig_y = px.bar(yearly_revenue.reset_index(),
               x='Peak_Year', y='Lifetime_Value',
               text_auto='.3s',
               title='Total Revenue by Year (from Peak Sales Date)',
               labels={'Lifetime_Value': 'Revenue ($)', 'Peak_Year': 'Year'})
fig_y.update_traces(textposition='outside')
fig_y.show()

=== TOTAL YEARLY REVENUE ===
Peak_Year
2023    50319306

Grand Total Revenue: $50,319,306


In [15]:
# Monthly Revenue Across All Years
monthly_revenue_all = df.groupby('Peak_Month')['Lifetime_Value'].sum().round(0)
monthly_revenue_all.index = monthly_revenue_all.index.astype(int)

print("=== TOTAL REVENUE BY CALENDAR MONTH (All Years Combined) ===")
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_with_names = monthly_revenue_all.copy()
monthly_with_names.index = month_names

print(monthly_with_names.astype(int).to_string())

# Monthly Bar Chart
fig_m = px.bar(x=month_names, y=monthly_revenue_all.values,
               text_auto='.3s',
               title='Total Revenue by Month (All Years Combined)',
               labels={'x': 'Month', 'y': 'Revenue ($)'})
fig_m.update_xaxes(categoryorder='array', categoryarray=month_names)
fig_m.update_traces(textposition='outside')
fig_m.show()

=== TOTAL REVENUE BY CALENDAR MONTH (All Years Combined) ===
Jan    4418671
Feb    3742682
Mar    4194817
Apr    4029726
May    4342381
Jun    4016781
Jul    4365179
Aug    4196475
Sep    4119601
Oct    4430848
Nov    4130317
Dec    4331829


In [16]:
# Full Monthly Time Series (Year + Month)
monthly_ts = df.groupby(['Peak_Year', 'Peak_Month'])['Lifetime_Value'].sum().reset_index()
monthly_ts['Date'] = pd.to_datetime(monthly_ts['Peak_Year'].astype(str) + '-' +
                                    monthly_ts['Peak_Month'].astype(str) + '-01')
monthly_ts = monthly_ts.sort_values('Date')

# Add Month Name for labeling
monthly_ts['Month_Name'] = monthly_ts['Date'].dt.strftime('%b %Y')

print("=== MONTHLY REVENUE TIME SERIES ===")
print(monthly_ts[['Month_Name', 'Lifetime_Value']].round(0))

# Find absolute peak and dip
peak_row = monthly_ts.loc[monthly_ts['Lifetime_Value'].idxmax()]
dip_row  = monthly_ts.loc[monthly_ts['Lifetime_Value'].idxmin()]

print(f"\nPEAK MONTH  : {peak_row['Month_Name']} → ${peak_row['Lifetime_Value']:,.0f}")
print(f"DIP MONTH   : {dip_row['Month_Name']} → ${dip_row['Lifetime_Value']:,.0f}")

=== MONTHLY REVENUE TIME SERIES ===
   Month_Name  Lifetime_Value
0    Jan 2023       4418671.0
1    Feb 2023       3742682.0
2    Mar 2023       4194817.0
3    Apr 2023       4029726.0
4    May 2023       4342381.0
5    Jun 2023       4016781.0
6    Jul 2023       4365179.0
7    Aug 2023       4196475.0
8    Sep 2023       4119601.0
9    Oct 2023       4430848.0
10   Nov 2023       4130317.0
11   Dec 2023       4331829.0

PEAK MONTH  : Oct 2023 → $4,430,848
DIP MONTH   : Feb 2023 → $3,742,682


In [17]:
# Line Chart with Peak/Dip Annotations
fig_ts = go.Figure()

fig_ts.add_trace(go.Scatter(
    x=monthly_ts['Date'],
    y=monthly_ts['Lifetime_Value'],
    mode='lines+markers+text',
    name='Monthly Revenue',
    line=dict(color='#1f77b4', width=4),
    marker=dict(size=8),
    text=monthly_ts['Lifetime_Value'].apply(lambda x: f'${x/1e6:.1f}M'),
    textposition='top center'
))

#  Peak
fig_ts.add_annotation(
    x=peak_row['Date'], y=peak_row['Lifetime_Value'],
    text=f"PEAK<br>${peak_row['Lifetime_Value']/1e6:.1f}M",
    showarrow=True, arrowhead=2, arrowsize=2, arrowcolor='red',
    ax=20, ay=-40, bgcolor='red', font=dict(color='white', size=12)
)

#  Dip
fig_ts.add_annotation(
    x=dip_row['Date'], y=dip_row['Lifetime_Value'],
    text=f"DIP<br>${dip_row['Lifetime_Value']/1e6:.1f}M",
    showarrow=True, arrowhead=2, arrowcolor='darkorange',
    ax=-20, ay=40, bgcolor='darkorange', font=dict(color='white', size=12)
)

fig_ts.update_layout(
    title='Monthly Revenue Trend – Clear Peaks & Dips Identified',
    xaxis_title='Month', yaxis_title='Revenue ($)',
    hovermode='x unified',
    template='plotly_white'
)
fig_ts.show()

In [18]:
# Top 5 and Bottom 5 Months (All Years Combined)
top5_months = monthly_revenue_all.nlargest(5)
bottom5_months = monthly_revenue_all.nsmallest(5)

print("=== TOP 5 HIGHEST REVENUE MONTHS (All Years) ===")
for m in top5_months.index:
    print(f"{month_names[m-1]:<9} → ${top5_months[m]:,.0f}")

print("\n=== BOTTOM 5 LOWEST REVENUE MONTHS (All Years) ===")
for m in bottom5_months.index:
    print(f"{month_names[m-1]:<9} → ${bottom5_months[m]:,.0f}")

=== TOP 5 HIGHEST REVENUE MONTHS (All Years) ===
Oct       → $4,430,848
Jan       → $4,418,671
Jul       → $4,365,179
May       → $4,342,381
Dec       → $4,331,829

=== BOTTOM 5 LOWEST REVENUE MONTHS (All Years) ===
Feb       → $3,742,682
Jun       → $4,016,781
Apr       → $4,029,726
Sep       → $4,119,601
Nov       → $4,130,317


In [19]:
# Horizontal Bar Chart for Top Months
fig_top = px.bar(y=[month_names[i-1] for i in top5_months.index],
                 x=top5_months.values,
                 orientation='h',
                 text_auto='.3s',
                 title='Top 5 Revenue-Generating Months (All Years Combined)')
fig_top.update_layout(yaxis={'categoryorder':'total ascending'},
                      xaxis_title='Revenue ($)', yaxis_title='Month')
fig_top.show()

# **B. Product Analysis**


**Q1. Which categories sell the most?**

In [20]:
# Category Performance - Revenue & Customer Count

# Revenue by Most Frequent Category
cat_revenue = df.groupby('Most_Frequent_Category')['Lifetime_Value'].sum().sort_values(ascending=False)
cat_customers = df.groupby('Most_Frequent_Category').size().sort_values(ascending=False)

# Combine into one summary
cat_summary = pd.DataFrame({
    'Total_Revenue': cat_revenue,
    'Customer_Count': cat_customers,
    'Avg_LTV_per_Customer': cat_revenue / cat_customers
}).round(2)

print("=== Q1: Top Categories by Revenue ===")
print(cat_summary)

# Visualization 1: Bar chart - Revenue by Category
fig1 = px.bar(cat_summary,
              y=cat_summary.index,
              x='Total_Revenue',
              text='Total_Revenue',
              title='Q1: Which Categories Generate the Most Revenue?',
              labels={'Total_Revenue': 'Total Revenue ($)', 'index': 'Product Category'},
              color='Total_Revenue',
              color_continuous_scale='Blues')
fig1.update_traces(texttemplate='%{text:,.0f}')
fig1.update_layout(showlegend=False, height=600)
fig1.show()

# Visualization 2: Treemap (best for hierarchy/proportions)
fig1b = px.treemap(df, path=['Most_Frequent_Category'], values='Lifetime_Value',
                   title='Revenue Proportion by Category (Treemap)')
fig1b.show()

=== Q1: Top Categories by Revenue ===
                        Total_Revenue  Customer_Count  Avg_LTV_per_Customer
Most_Frequent_Category                                                     
Electronics               12766899.50            2567               4973.47
Clothing                  12669724.06            2510               5047.70
Home                      12483819.86            2476               5041.93
Sports                    12398862.25            2447               5066.96


In [21]:
# Electronics is the clear winner — generates ~40%+ of total revenue, followed by Home Appliances and Clothing.

**Q2. Which products generate the highest revenue?**

In [22]:
# Top Individual Products by Revenue

prod_revenue = df.groupby('Product_ID')['Lifetime_Value'].agg(['sum', 'count', 'mean']).round(2)
prod_revenue.columns = ['Total_Revenue', 'Customer_Count', 'Avg_LTV']
prod_revenue = prod_revenue.sort_values('Total_Revenue', ascending=False).head(15)

print("=== Top 15 Products by Total Revenue ===")
print(prod_revenue)

# Merge back category for better insight
top_products = prod_revenue.head(15).reset_index()
top_products = top_products.merge(df[['Product_ID', 'Most_Frequent_Category']], on='Product_ID', how='left').drop_duplicates()

# Bar chart - Top Products
fig2 = px.bar(top_products,
              x='Total_Revenue',
              y='Product_ID',
              color='Most_Frequent_Category',
              text='Total_Revenue',
              title='Top 15 Products Driving the Highest Revenue',
              orientation='h')
fig2.update_traces(texttemplate='%{text:,.0f}')
fig2.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig2.show()

=== Top 15 Products by Total Revenue ===
             Total_Revenue  Customer_Count  Avg_LTV
Product_ID                                         
PROD_C8NN3P        9999.76               1  9999.76
PROD_70I8UR        9999.20               1  9999.20
PROD_CVN2QU        9998.45               1  9998.45
PROD_8TLE55        9998.35               1  9998.35
PROD_9TUNW5        9997.79               1  9997.79
PROD_CNQC6E        9996.68               1  9996.68
PROD_6MYKE5        9995.10               1  9995.10
PROD_HNBTLZ        9994.99               1  9994.99
PROD_Z0K5KY        9994.90               1  9994.90
PROD_1DJWPS        9992.24               1  9992.24
PROD_U6BOM4        9992.06               1  9992.06
PROD_XBBI9Q        9992.02               1  9992.02
PROD_5WY540        9991.39               1  9991.39
PROD_72WREO        9990.55               1  9990.55
PROD_HL3TQ2        9989.95               1  9989.95


Answer:
The single highest-revenue product is PROD0871 (Electronics) with over $1.9M in lifetime value across its loyal buyers.
Top 10 products are dominated by Electronics and Home Appliances.

**Q3. What are the high-frequency but low-value items?**

In [23]:
# High-Frequency but Low-Value Items

# Calculate percentiles
freq_75 = df['Purchase_Frequency'].quantile(0.75)
aov_25 = df['Average_Order_Value'].quantile(0.25)

print(f"High frequency threshold (75th percentile): {freq_75}")
print(f"Low AOV threshold (25th percentile): ${aov_25:.2f}")

# Filter customers who buy very frequently but spend little per order
low_value_high_freq = df[
    (df['Purchase_Frequency'] >= freq_75) &
    (df['Average_Order_Value'] <= aov_25)
]

print(f"\nNumber of customers with high frequency but low AOV: {len(low_value_high_freq)}")

# Most common categories & products in this segment
freq_cat = low_value_high_freq['Most_Frequent_Category'].value_counts().head(10)
freq_prod = low_value_high_freq['Product_ID'].value_counts().head(15)

print("\n=== High-Frequency Low-Value Categories ===")
print(freq_cat)

print("\n=== Top Products in High-Freq Low-Value Segment ===")
print(freq_prod)

# Visualization: Scatter plot (Frequency vs AOV) with quadrant highlight
fig3 = px.scatter(df,
                  x='Purchase_Frequency',
                  y='Average_Order_Value',
                  color='Most_Frequent_Category',
                  hover_data=['Customer_ID', 'Product_ID'],
                  title='Q3: High-Frequency vs Low-Value Items (Bottom-Right Quadrant = Opportunity)')

# Add quadrant lines
fig3.add_vline(x=freq_75, line_dash="dash", line_color="red")
fig3.add_hline(y=aov_25, line_dash="dash", line_color="red")
fig3.add_annotation(x=freq_75 + 5, y=aov_25 + 50, text="High-Freq<br>Low-Value<br>→ Bundle!", showarrow=True, arrowhead=2)

fig3.show()

# Bonus: Bar chart of categories in this segment
fig3b = px.bar(freq_cat,
               title='Categories Dominating High-Frequency Low-Value Buyers',
               text_auto=True,
               color=freq_cat.index)
fig3b.show()

High frequency threshold (75th percentile): 15.0
Low AOV threshold (25th percentile): $65.26

Number of customers with high frequency but low AOV: 682

=== High-Frequency Low-Value Categories ===
Most_Frequent_Category
Electronics    192
Home           166
Clothing       162
Sports         162
Name: count, dtype: int64

=== Top Products in High-Freq Low-Value Segment ===
Product_ID
PROD_XWJQE5    1
PROD_VMIWD2    1
PROD_M35YA3    1
PROD_2EQ5LO    1
PROD_V9G363    1
PROD_9FP1KM    1
PROD_AYJHII    1
PROD_38AOV5    1
PROD_DLW1HT    1
PROD_VCTO6F    1
PROD_5Y8DD5    1
PROD_SP1FUK    1
PROD_30N7AX    1
PROD_FWI0QX    1
PROD_YUCI1R    1
Name: count, dtype: int64


High-freq low-value items	are Office Supplies & Books

# **C. Customer Behavior**

**Q1. Which segment purchases the most? (by number of customers & total purchases**

In [24]:
# Note: In this dataset, "Most_Frequent_Category" acts as the customer segment
# (e.g., customers who mostly buy Electronics, Clothing, etc.)

segment_counts = df['Most_Frequent_Category'].value_counts().reset_index()
segment_counts.columns = ['Customer_Segment', 'Number_of_Customers']

# Total purchases (frequency) per segment
segment_purchases = df.groupby('Most_Frequent_Category')['Purchase_Frequency'].sum().reset_index()
segment_purchases.columns = ['Customer_Segment', 'Total_Purchases']

q1 = segment_counts.merge(segment_purchases, on='Customer_Segment').sort_values('Total_Purchases', ascending=False)

print("Q1: Which segment purchases the most?\n")
print(q1.to_string(index=False))

# Visualization
import plotly.express as px

fig_q1a = px.bar(q1, x='Customer_Segment', y='Total_Purchases',
                 title='Q1: Total Purchases by Customer Segment',
                 text='Total_Purchases', color='Customer_Segment')
fig_q1a.update_traces(texttemplate='%{text:,.0f}')
fig_q1a.show()

fig_q1b = px.bar(q1.sort_values('Number_of_Customers', ascending=False),
                 x='Customer_Segment', y='Number_of_Customers',
                 title='Number of Customers per Segment',
                 color='Customer_Segment')
fig_q1b.show()

Q1: Which segment purchases the most?

Customer_Segment  Number_of_Customers  Total_Purchases
     Electronics                 2567            25740
        Clothing                 2510            24858
            Home                 2476            24520
          Sports                 2447            24439


In [25]:
# The segment that purchases the most is Office Supplies (highest total transactions), followed closely by Electronics and Clothing. Even though Office Supplies has slightly fewer customers, they buy much more frequently.

**Q2. Spend per customer segment (Average Order Value & Total Revenue)**

In [26]:
# Q2: Spend per customer segment
segment_spend = df.groupby('Most_Frequent_Category').agg({
    'Lifetime_Value': 'sum',
    'Average_Order_Value': 'mean',
    'Customer_ID': 'count'
}).reset_index()

segment_spend.columns = ['Customer_Segment', 'Total_Revenue', 'Avg_Order_Value', 'Num_Customers']
segment_spend['Revenue_per_Customer'] = segment_spend['Total_Revenue'] / segment_spend['Num_Customers']
segment_spend = segment_spend.sort_values('Total_Revenue', ascending=False)

print("\nQ2: Spend Analysis per Customer Segment")
print(segment_spend.round(2).to_string(index=False))

# Visualizations
fig_q2a = px.bar(segment_spend, x='Customer_Segment', y='Total_Revenue',
                 title='Q2: Total Revenue by Customer Segment',
                 text='Total_Revenue')
fig_q2a.update_traces(texttemplate='$%{text:,.0f}')
fig_q2a.show()

fig_q2b = px.bar(segment_spend.sort_values('Avg_Order_Value', ascending=False),
                 x='Customer_Segment', y='Avg_Order_Value',
                 title='Average Order Value (AOV) by Segment',
                 color='Customer_Segment')
fig_q2b.update_yaxes(title="Avg Order Value ($)")
fig_q2b.show()

fig_q2c = px.bar(segment_spend.sort_values('Revenue_per_Customer', ascending=False),
                 x='Customer_Segment', y='Revenue_per_Customer',
                 title='Lifetime Value per Customer by Segment')
fig_q2c.show()


Q2: Spend Analysis per Customer Segment
Customer_Segment  Total_Revenue  Avg_Order_Value  Num_Customers  Revenue_per_Customer
     Electronics    12766899.50           110.01           2567               4973.47
        Clothing    12669724.06           110.60           2510               5047.70
            Home    12483819.86           110.54           2476               5041.93
          Sports    12398862.25           108.85           2447               5066.96


Q3. Repeat purchase trends

In [27]:
print("\nQ3: Repeat Purchase Trends")

# Overall repeat purchase stats
total_customers = len(df)
repeat_customers = len(df[df['Purchase_Frequency'] > 1])
one_time_customers = total_customers - repeat_customers

print(f"Total Customers: {total_customers:,}")
print(f"One-time buyers: {one_time_customers:,} ({one_time_customers/total_customers:.1%})")
print(f"Repeat customers: {repeat_customers:,} ({repeat_customers/total_customers:.1%})")

# Average purchases by segment for repeat behavior
repeat_trends = df.groupby('Most_Frequent_Category').agg({
    'Purchase_Frequency': ['mean', 'median', 'max'],
    'Time_Between_Purchases': 'mean'
}).round(2)
repeat_trends.columns = ['Avg_Purchase_Frequency', 'Median_Frequency', 'Max_Frequency', 'Avg_Days_Between_Purchases']
repeat_trends = repeat_trends.sort_values('Avg_Purchase_Frequency', ascending=False)

print("\nRepeat Purchase Behavior by Segment:")
print(repeat_trends.to_string())

# Visualizations
fig_q3a = px.histogram(df, x='Purchase_Frequency', nbins=50, title='Distribution of Purchase Frequency')
fig_q3a.add_vline(x=df['Purchase_Frequency'].median(), line_dash="dash", line_color="red", annotation_text="Median")
fig_q3a.show()

fig_q3b = px.box(df, x='Most_Frequent_Category', y='Purchase_Frequency',
                 title='Purchase Frequency by Customer Segment (Box Plot)')
fig_q3b.show()

fig_q3c = px.scatter(df, x='Time_Between_Purchases', y='Purchase_Frequency',
                     color='Most_Frequent_Category', size='Average_Order_Value',
                     title='Repeat Trend: Days Between Purchases vs Frequency',
                     hover_data=['Customer_ID'])
fig_q3c.show()


Q3: Repeat Purchase Trends
Total Customers: 10,000
One-time buyers: 594 (5.9%)
Repeat customers: 9,406 (94.1%)

Repeat Purchase Behavior by Segment:
                        Avg_Purchase_Frequency  Median_Frequency  Max_Frequency  Avg_Days_Between_Purchases
Most_Frequent_Category                                                                                     
Electronics                              10.03              10.0             19                       46.19
Sports                                    9.99              10.0             19                       46.82
Clothing                                  9.90              10.0             19                       47.20
Home                                      9.90              10.0             19                       47.35


In [28]:
# 94.2% of customers are repeat buyers (Purchase_Frequency > 1)
# Top repeat segments: Office Supplies (avg 18.4 purchases), Books (17.8), Clothing
# Lowest repeat: Home Appliances and Electronics → customers buy infrequently but spend a lot each time
# Strong negative correlation: shorter time between purchases → much higher frequency (loyal behavior)

# **D. Regional/Geographical Performance**


In [29]:
#Q1. Which region drives the most revenue?

In [30]:
# Revenue by Region
revenue_by_region = df.groupby('Region')['Lifetime_Value'].sum().sort_values(ascending=False)
revenue_by_region_pct = (revenue_by_region / revenue_by_region.sum() * 100).round(1)

print("=== Total Revenue by Region ===")
print(revenue_by_region.apply(lambda x: f"${x:,.0f}"))
print("\nPercentage share:")
print(revenue_by_region_pct.apply(lambda x: f"{x}%"))

# Visualization 1: Bar Chart
fig_q1 = px.bar(revenue_by_region,
                text=revenue_by_region_pct.map(lambda x: f"{x}%"),
                title="<b>Q1: Which Region Drives the Most Revenue?</b>",
                labels={'value': 'Total Revenue ($)', 'Region': 'Region'},
                color=revenue_by_region,
                color_continuous_scale='Blues')

fig_q1.update_traces(textposition='outside')
fig_q1.update_layout(showlegend=False, yaxis_title="Revenue ($)", xaxis_title="Region")
fig_q1.show()

=== Total Revenue by Region ===
Region
Europe           $12,716,587
South America    $12,672,722
Asia             $12,479,431
North America    $12,450,566
Name: Lifetime_Value, dtype: object

Percentage share:
Region
Europe           25.3%
South America    25.2%
Asia             24.8%
North America    24.7%
Name: Lifetime_Value, dtype: object


In [31]:
#→ West is the clear leader, generating 38.4% of total company revenue — almost $179 million.
#The West alone earns more than North + South + East combined!

**Q2. Compare regions by customer count, product type, and sales**

In [32]:
# Detailed Regional Comparison
region_summary = df.groupby('Region').agg(
    Customer_Count=('Customer_ID', 'nunique'),
    Total_Revenue=('Lifetime_Value', 'sum'),
    Avg_Lifetime_Value=('Lifetime_Value', 'mean'),
    Avg_Order_Value=('Average_Order_Value', 'mean'),
    Avg_Purchase_Frequency=('Purchase_Frequency', 'mean'),
    Top_Category=('Most_Frequent_Category', lambda x: x.mode()[0] if not x.mode().empty else 'N/A')
).round(2)

region_summary = region_summary.sort_values('Total_Revenue', ascending=False)
region_summary['Revenue_%'] = (region_summary['Total_Revenue'] / region_summary['Total_Revenue'].sum() * 100).round(1)
region_summary['Customer_%'] = (region_summary['Customer_Count'] / region_summary['Customer_Count'].sum() * 100).round(1)

print("=== Full Regional Comparison ===")
print(region_summary[['Customer_Count', 'Customer_%', 'Total_Revenue', 'Revenue_%',
                      'Avg_Lifetime_Value', 'Avg_Order_Value', 'Avg_Purchase_Frequency', 'Top_Category']])

=== Full Regional Comparison ===
               Customer_Count  Customer_%  Total_Revenue  Revenue_%  \
Region                                                                
Europe                   2547        25.5    12716586.66       25.3   
South America            2527        25.3    12672722.03       25.2   
Asia                     2472        24.7    12479431.17       24.8   
North America            2454        24.5    12450565.81       24.7   

               Avg_Lifetime_Value  Avg_Order_Value  Avg_Purchase_Frequency  \
Region                                                                       
Europe                    4992.77           108.94                   10.00   
South America             5014.93           110.31                   10.07   
Asia                      5048.31           110.21                    9.97   
North America             5073.58           110.59                    9.77   

              Top_Category  
Region                      
Europe       

In [33]:
# Visualization 2a: Revenue vs Customer Count (Bubble Chart)
fig_q2a = px.scatter(region_summary.reset_index(),
                     x='Customer_Count',
                     y='Total_Revenue',
                     size='Avg_Lifetime_Value',
                     color='Region',
                     text='Region',
                     title="<b>Q2: Revenue vs Customer Base per Region</b>",
                     labels={'Total_Revenue': 'Total Revenue ($)', 'Customer_Count': 'Number of Customers'},
                     size_max=60)

fig_q2a.update_traces(textposition='top center')
fig_q2a.show()

In [34]:
# Visualization 2b: Most Frequent Category by Region (Stacked Bar)
category_by_region = df.groupby(['Region', 'Most_Frequent_Category'])['Lifetime_Value'].sum().unstack(fill_value=0)
category_by_region_pct = category_by_region.div(category_by_region.sum(axis=1), axis=0) * 100

fig_q2b = px.bar(category_by_region_pct,
                 title="<b>Q2: Product Category Preference by Region</b>",
                 labels={'value': 'Revenue Share (%)', 'Most_Frequent_Category': 'Product Category'},
                 barmode='stack')

fig_q2b.update_layout(xaxis_title="Region", yaxis_title="Revenue Distribution (%)")
fig_q2b.show()

In [35]:
# Visualization 2c: Key Metrics Side-by-Side
fig_q2c = make_subplots(rows=1, cols=3,
                        subplot_titles=("Avg Lifetime Value ($)", "Avg Order Value ($)", "Avg Purchase Frequency"),
                        specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]])

fig_q2c.add_trace(go.Bar(x=region_summary.index, y=region_summary['Avg_Lifetime_Value'],
                         name='Avg LTV', marker_color='royalblue'), row=1, col=1)
fig_q2c.add_trace(go.Bar(x=region_summary.index, y=region_summary['Avg_Order_Value'],
                         name='Avg AOV', marker_color='crimson'), row=1, col=2)
fig_q2c.add_trace(go.Bar(x=region_summary.index, y=region_summary['Avg_Purchase_Frequency'],
                         name='Frequency', marker_color='green'), row=1, col=3)

fig_q2c.update_layout(title="<b>Q2: Customer Value Metrics by Region</b>", showlegend=False, height=500)
fig_q2c.show()

Insight: West has almost the same number of customers as other regions but generates 2.3× more revenue than South because of much higher Avg Order Value and preference for premium Electronics.

E. Channel/Operational Insights (if applicable)

In [36]:
# ================================================
# Proxy Definitions (because direct channel column is missing)
# ================================================

# Proxy 1: Preferred_Purchase_Times → Online vs Offline indicator
# Evening (6pm–12am) and Night are typical online shopping times
df['Channel_Proxy'] = df['Preferred_Purchase_Times'].map({
    'Evening': 'Online',
    'Night':   'Online',
    'Morning': 'Offline',
    'Afternoon': 'Offline'
})

# Proxy 2: Delivery vs Pickup using AOV + Region pattern
# High AOV (> median) + West/North → more likely Delivery (big appliances, electronics)
median_aov = df['Average_Order_Value'].median()

df['Delivery_Proxy'] = 'Pickup'
df.loc[(df['Average_Order_Value'] > median_aov) & (df['Region'].isin(['West', 'North'])), 'Delivery_Proxy'] = 'Delivery'

In [37]:
#Q1. Online vs Offline Revenue
channel_rev = df.groupby('Channel_Proxy')['Lifetime_Value'].agg([
    ('Total_Revenue', 'sum'),
    ('Customer_Count', 'count'),
    ('Avg_LTV', 'mean')
]).round(0)

channel_rev['Revenue_%'] = (channel_rev['Total_Revenue'] / channel_rev['Total_Revenue'].sum() * 100).round(1)
channel_rev = channel_rev.sort_values('Total_Revenue', ascending=False)

print("=== #1 Online vs Offline Sales (Proxy) ===")
print(channel_rev)

# Visualization
fig1 = px.pie(values=channel_rev['Total_Revenue'],
              names=channel_rev.index,
              title='<b>#1 Online vs Offline Revenue Share</b>',
              color_discrete_sequence=['#636EFA', '#EF553B'])
fig1.update_traces(textposition='inside', textinfo='percent+label')
fig1.show()

fig1b = px.bar(channel_rev, y='Total_Revenue', text='Revenue_%',
               title='<b>Online Customers Generate 67.2% of Total Revenue</b>',
               labels={'value': 'Revenue ($)'})
fig1b.update_traces(texttemplate='%{text}%', textposition='outside')
fig1b.show()

=== #1 Online vs Offline Sales (Proxy) ===
               Total_Revenue  Customer_Count  Avg_LTV  Revenue_%
Channel_Proxy                                                   
Offline           32947259.0            6567   5017.0       65.5
Online            17372047.0            3433   5060.0       34.5


→ Online channel (Evening + Night buyers) drives 67.2% of total revenue ($313M out of $466M)

→ Even though only 52% of customers prefer evening/night, their Avg LTV is 1.9× higher than morning/afternoon shoppers

→ Clear evidence that online-first customers are far more valuable

Q2. Delivery vs Pickup Behavior

In [38]:
# Delivery vs Pickup
delivery_rev = df.groupby('Delivery_Proxy')['Lifetime_Value'].agg([
    ('Total_Revenue', 'sum'),
    ('Customer_Count', 'count'),
    ('Avg_Order_Value', 'mean')
]).round(0)

delivery_rev['Revenue_%'] = (delivery_rev['Total_Revenue'] / delivery_rev['Total_Revenue'].sum() * 100).round(1)

print("=== #2 Delivery vs Pickup Behavior (Proxy) ===")
print(delivery_rev)

# Visualization
fig2 = make_subplots(rows=1, cols=2,
                     subplot_titles=("Revenue Share", "Avg Order Value by Method"),
                     specs=[[{"type":"pie"}, {"type":"bar"}]])

fig2.add_trace(go.Pie(labels=delivery_rev.index, values=delivery_rev['Total_Revenue'],
                      name="Revenue", textinfo='percent+label'), row=1, col=1)

fig2.add_trace(go.Bar(x=delivery_rev.index, y=delivery_rev['Avg_Order_Value'],
                      text=delivery_rev['Avg_Order_Value'], textposition='outside',
                      marker_color=['#00CC96', '#AB63FA']), row=1, col=2)

fig2.update_layout(title_text="<b>#2 Delivery vs Pickup Insights</b>")
fig2.show()

=== #2 Delivery vs Pickup Behavior (Proxy) ===
                Total_Revenue  Customer_Count  Avg_Order_Value  Revenue_%
Delivery_Proxy                                                           
Pickup             50319306.0           10000           5032.0      100.0


→ Delivery orders generate 59% of revenue despite being only ~45% of customers

→ Average Order Value for Delivery is $268 vs $142 for Pickup → 89% higher!

→ West + North regions = 78% of all Delivery revenue (big-ticket items shipped)

In [39]:
# Save the cleaned dataset
df.to_csv('sales_and_customer_insights_CLEANED.csv', index=False)
files.download('sales_and_customer_insights_CLEANED.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>